# Lab 1: Vacuum field optimisation

Let's begin by installing the necessary dependencies

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade numpy scipy
!{sys.executable} -m pip install plotly pandas
!{sys.executable} -m pip install -vvv --no-cache-dir git+https://github.com/florianwechsung/PyPlasmaOpt@fw/riskaverse

In [ ]:
from pyplasmaopt import *

Now we can start by setting up a first simple optimisation problem

In [ ]:
nfp = 2 # number of rotations
ppp = 20 # number of quadrature points per fourier period
# let's get some initial data
(coils, currents, expansion_axis, eta_bar) = get_24_coil_data(nfp=nfp, ppp=ppp, at_optimum=False)
# and now let's turn this into a Stellarator
stellarator = CoilCollection(coils, currents, nfp, True)

In [ ]:
# Specify a target iota and target lengths for the coils and the axis
iota_target = 0.103
coil_length_target = 4.398229715025710
magnetic_axis_length_target = 6.356206812106860
# and lastly, let's pick an initial guess for eta_bar
eta_bar = -2.25

Now we're ready to build our objective

In [ ]:
obj = SimpleNearAxisQuasiSymmetryObjective(
        stellarator, expansion_axis, iota_target, eta_bar=eta_bar,
        coil_length_target=coil_length_target, magnetic_axis_length_target=magnetic_axis_length_target)

The objective can be evaluated at a point `x` by calling
    
    obj.update(x)


In [ ]:
x = obj.x0
obj.update(x)
print('f(x) =', obj.res)
print('gradf(x) =', obj.dres)

We can plot the coils of stellarator and the expansion axis using the `plot_stellarator` function.

In [ ]:
plot_stellarator(stellarator, axis=expansion_axis)

## Tasks
After calling `obj.update(x)` the objective value at `x` is stored in `obj.res` and the gradient is stored in `obj.dres`.

1) Look at the documentation for the `minimize` function in `scipy.optimize` and use the `BFGS` algorithm to solve the optimisation problem. You can use the `obj.callback()` function to obtain output as the optimisation progresses.

2) Plot the resulting coils. How do they differ from the initial coils?

3) Plot the objective function throughout the optimisation.

## Solution

### Task 1

In [ ]:
from scipy.optimize import minimize

Jvals = []

def scipy_fun(x):
    obj.update(x)
    res = obj.res
    Jvals.append(res)
    dres = obj.dres
    return res, dres

res = minimize(scipy_fun, x, jac=True, method='bfgs', tol=1e-20,
               options={"maxiter": 200},
               callback=obj.callback)

### Task 2

In [ ]:
plot_stellarator(stellarator, axis=expansion_axis)

### Task 3

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.semilogy(Jvals)

Note: is you use the callback as above, the function evaluations at the _accepted_ optimisation iterates are stored in `obj.Jvals`. That way you obtain a monotone curve in the plot. Before running a new optimisation problem, you can clear this array by calling `obj.clear_history()`.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.semilogy(obj.Jvals)